<a href="https://colab.research.google.com/github/Daryamn88/Python_Assignmnets/blob/main/term_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Darya Mansouri - 101394900
#Term Project

import sqlite3

class Product:
    def __init__(self, product_id, name, price, category):
        self.product_id = product_id
        self.name = name
        self.price = price
        self.category = category

class Order:
    def __init__(self):
        self.items = []

    def add_item(self, product, quantity):
        self.items.append({"product": product, "quantity": quantity})

    def calculate_total(self):
        total = 0
        for item in self.items:
            price = item["product"].price
            quantity = item["quantity"]
            if quantity >= 10:
                price *= 0.95  # Apply 5% discount for bulk purchase
            total += price * quantity
        return total

class OnlineStore:
    def __init__(self):
        self.connection = sqlite3.connect('store.db')
        self.cursor = self.connection.cursor()
        self.create_tables()
        self.cart = Order()  # Initialize the cart attribute

    def create_tables(self):
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS products (
                                id INTEGER PRIMARY KEY,
                                name TEXT,
                                price REAL,
                                category TEXT)''')
        self.connection.commit()

    def add_product(self, product):
        self.cursor.execute("INSERT INTO products (name, price, category) VALUES (?, ?, ?)",
                            (product.name, product.price, product.category))
        self.connection.commit()
        print("Product added successfully.")

    def display_products(self):
        while True:
            self.cursor.execute("SELECT DISTINCT category FROM products")
            categories = self.cursor.fetchall()
            for category in categories:
                category_name = category[0]
                print(f"\n{category_name}:\n")
                self.cursor.execute("SELECT * FROM products WHERE category=?", (category_name,))
                products = self.cursor.fetchall()
                print("Item ID       | Item Name          | Price")
                for product in products:
                    product_id, name, price, _ = product
                    print(f"| {product_id:<11} | {name:<18} | ${price:<13} |")

            choice = input("\nEnter the ID of the product you want to add to cart (or type 'no' to stop): ")
            if choice.lower() == 'no':
                break
            elif choice:
                product_id = int(choice)
                quantity = int(input("Enter quantity: "))
                # Fetch product details from the database
                self.cursor.execute("SELECT * FROM products WHERE id=?", (product_id,))
                product_data = self.cursor.fetchone()
                if product_data:
                    product = Product(*product_data)
                    self.cart.add_item(product, quantity)
                    print("Product added to cart.")
                else:
                    print("Product not found.")



    def view_cart(self):
        print("\nYour Cart:")
        for item in self.cart.items:
            print(f"Product: {item['product'].name}, Quantity: {item['quantity']}")
        total_price = self.cart.calculate_total()
        print(f"Total Price: ${total_price:.2f}")

    def checkout(self):
        print("\nYour Cart:")
        self.view_cart()
        confirm = input("Confirm your order (yes/no): ").lower()
        if confirm == "yes":
            total_price = self.cart.calculate_total()
            print(f"Total Price: ${total_price:.2f}")
            if total_price < 75:
                total_price += 9.99  # Shipping cost
            total_price *= 1.13  # Apply 13% taxes
            print(f"Total Price including taxes and shipping: ${total_price:.2f}")
            print("Thank you for confirming your order!")
            self.cart = Order()  # Clear the cart after successful confirmation
        else:
            continue_shopping = input("Do you want to continue shopping? (yes/no): ").lower()
            if continue_shopping == "no":
                print("Thank You For Using This Online Store.")
                exit()
            else:
                print("Returning to main menu.")
                return


    def close(self):
        self.connection.close()

def main():
    store = OnlineStore()
    while True:
        print("\nWelcome to the Online Store")
        menu_items = ["1. View Products",
                      "",
                      "2. Add Product",
                      "",
                      "3. Checkout",
                      "",
                      "4. Exit"]
        print("-" * 95)
        for item in menu_items:
            print("| {:<90} |".format(item))
        print("-" * 95)
        choice = input("Enter your choice: ")

        if choice == "1":
            print("\nProducts available:")
            store.display_products()

        elif choice == "2":
            name = input("Enter product name: ")
            price = float(input("Enter product price: "))
            category = input("Enter product category: ")
            product = Product(None, name, price, category)
            store.add_product(product)

        elif choice == "3":
            store.checkout()
            continue_shopping = input("Do you want to continue shopping? (yes/no): ").lower()
            if continue_shopping == "no":
              print("Thank You For Using This Online Store.")
              break

        elif choice == "4":
            break

        else:
            print("Invalid choice. Please try again.")

    store.close()

if __name__ == "__main__":
    main()



Welcome to the Online Store
-----------------------------------------------------------------------------------------------
| 1. View Products                                                                           |
|                                                                                            |
| 2. Add Product                                                                             |
|                                                                                            |
| 3. Checkout                                                                                |
|                                                                                            |
| 4. Exit                                                                                    |
-----------------------------------------------------------------------------------------------
Enter your choice: 1

Products available:

electronics:

Item ID       | Item Name          | Price
| 1           